In [1]:
# Import dependencies
import pandas as pd
import sqlalchemy
from pprint import pprint
from flask import Flask, jsonify
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import create_engine, func

In [2]:
# Create engine to wardata.sqlite
engine = create_engine("sqlite:///data/wardata.sqlite")
DBSession = sessionmaker(bind=engine)
session = DBSession()

In [3]:
Base = declarative_base()
class Baseball(Base):
    __tablename__ = "baseball"
    player_id = Column(Integer, primary_key=True)
    rank_in_country = Column(Integer)
    player_name = Column(String)
    war = Column(Float)
    year_min = Column(Integer)
    year_max = Column(Integer)
    age_range = Column(String)
    birth_location = Column(String)
    country = Column(String)
    position = Column(String)

In [5]:
variable = session.query(Baseball)

#Sample Query
new_df = pd.read_sql("SELECT * FROM baseball WHERE year_min > 1914", engine)
new_df

,player_id,rank_in_country,player_name,war,year_min,year_max,age_range,birth_location,country,position
0,0,1,Barry Bonds,162.8,1986,2007,21-42,"Riverside, CA",USA,Hitter
1,2,3,Willie Mays,156.1,1948,1973,17-42,"Westfield, AL",USA,Hitter
2,4,5,Henry Aaron,143.0,1954,1976,20-42,"Mobile, AL",USA,Hitter
3,7,8,Stan Musial,128.6,1941,1963,20-42,"Donora, PA",USA,Hitter
4,8,9,Rogers Hornsby,127.3,1915,1937,19-41,"Winters, TX",USA,Hitter
...,...,...,...,...,...,...,...,...,...,...
2338,2747,87,Héctor Fajardo,-1.1,1991,1995,20-24,"Sahuayo, MEX",MEX,Pitcher
2339,2749,89,José Peña,-1.2,1969,1972,26-29,"Ciudad Juarez, MEX",MEX,Pitcher
2340,2751,91,Luis Mendoza,-1.2,2007,2013,23-29,"Veracruz, MEX",MEX,Pitcher
2341,2752,92,Edgar González,-1.3,2003,2013,20-30,"San Nicolas de los Garza, MEX",MEX,Pitcher


In [8]:
for record in new_df["player_id"]:
    print(record)

0
2
4
7
8
10
11
12
13
14
15
16
18
19
20
21
22
24
25
26
27
28
30
32
33
34
36
37
38
39
40
41
42
43
46
47
48
49
50
51
53
54
55
56
57
58
59
61
62
64
65
66
67
68
69
70
71
72
73
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
93
97
99
100
102
103
105
106
107
108
109
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
137
138
139
140
141
143
144
145
149
151
153
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
173
174
175
176
177
178
179
181
182
183
184
185
186
189
192
194
195
196
198
199
200
201
202
203
204
206
207
208
209
210
212
216
217
221
222
224
228
229
231
234
235
237
238
241
244
247
248
250
251
252
253
254
256
263
264
265
270
271
272
274
275
277
278
279
280
281
282
284
285
287
290
291
292
298
299
300
301
302
303
305
306
307
309
310
313
317
318
319
321
323
325
328
331
333
338
347
348
350
353
354
356
358
359
361
362
364
365
366
370
373
374
376
378
382
386
388
391
394
395
396
399
400
401
402
403
404
405
406
407
408
409
